import

In [1]:
import torch
from torch import nn, optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import r2_score

load dataset

In [2]:
data_frame_train = pd.read_csv('/content/train.csv')
date_frame_test = pd.read_csv('/content/test.csv')

train

In [3]:
x_train = data_frame_train.drop('MedHouseVal', axis=1)
y_train = data_frame_train['MedHouseVal']

test

In [4]:
x_test = date_frame_test.drop('MedHouseVal', axis=1)
y_test = date_frame_test['MedHouseVal']

# **preprocessing**

# **`convert to tensor`**

train

In [5]:
x_train = torch.tensor(x_train.values, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.float32)

test

In [6]:
x_test = torch.FloatTensor(x_test.values)
y_test = torch.FloatTensor(y_test.values)

normalization

In [7]:
mu = x_train.mean(dim=0)
std = x_train.std(dim=0)

In [8]:
x_train = (x_train - mu) / std

Dataloader

In [9]:
train_dataset = TensorDataset(x_train, y_train)

In [10]:
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)

In [11]:
test_dataset = TensorDataset(x_test, y_test)

In [12]:
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False)

define model

In [24]:
num_feats = 8
out_feats = 1
hiden_layer_one = 64
hiden_layer_two = 32

model = nn.Sequential(nn.Linear(num_feats, hiden_layer_one),
              nn.ReLU(),
              nn.Linear(hiden_layer_one, hiden_layer_two),
              nn.ReLU(),
              nn.Linear(hiden_layer_two, out_feats))

loss and optimizer

In [25]:
loss_function = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

train loop

In [ ]:
num_epochs = 400
for epoch in range(num_epochs):
    loss_train = 0

    for x_batch, y_batch in train_loader:
        yp = model(x_batch)
        loss = loss_function(yp.squeeze(), y_batch)
        loss_train += loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    loss_test = 0
    for x_batch, y_batch in test_loader:
        yp = model(x_batch)
        loss_test += loss_function(yp.squeeze(), y_batch)

    loss_total_train = loss_train / len(train_loader)
    loss_total_test = loss_test / len(test_loader)
    if epoch % 10 == 0:
        print("epoch:", epoch)
        print(f"loss_total_train = {loss_total_train:.4}")
        print(f"loss_total_test = {loss_total_test:.4}")
        print()

learning curve

prediction

In [ ]:
yp_total = []
yt_total = []

with torch.no_grad():
    for x, y in test_loader:
        yp = model(x)
        yp_total.append(yp.squeeze())
        yt_total.append(y)



In [ ]:
yp_total = torch.cat(yp_total)
yt_total = torch.cat(yt_total)

In [ ]:
r2_score(yp_total, yt_total)